In [1]:
import os
from settings_v2 import get_pkl_path
import joblib


def test(model_pkl,v_pkl):
# Load model and vectorizer
    nb = joblib.load( os.path.join(get_pkl_path(), model_pkl))
    vectorizer = joblib.load(os.path.join(get_pkl_path(),v_pkl ))

# Example email input
    email_subject = "Important Update"
    email_body = "Click here to verify your account immediately."
    email_text = email_subject + " " + email_body

# Transform and predict
    X_input = vectorizer.transform([email_text])
    prediction = nb.predict(X_input)
    confidence = nb.predict_proba(X_input)
    print(f"The label is {prediction[0]}, the confidence is  {confidence[0][1]}")


In [2]:
test(f'naive_bayes_model_final_bow.pkl',f'bow_vectorizer_final.pkl')

The label is 1, the confidence is  0.9999999999690345


In [4]:
import os
import joblib
import shap
import numpy as np
import matplotlib.pyplot as plt
from settings_v2 import get_pkl_path

def test_with_shap(model_pkl, v_pkl,uid):
    # Load model and vectorizer
    model = joblib.load(os.path.join(get_pkl_path(), model_pkl))
    vectorizer = joblib.load(os.path.join(get_pkl_path(), v_pkl))

    # Example email input
    email_subject = "Important Update"
    email_body = "Click here to verify your account immediately."
    email_text = email_subject + " " + email_body

    # Preprocess
    X_input = vectorizer.transform([email_text])

    # Prediction
    prediction = model.predict(X_input)
    confidence = model.predict_proba(X_input)
    print(f"\nPrediction: {prediction[0]} | Phishing Confidence: {confidence[0][1]:.4f}")
    from sklearn.pipeline import make_pipeline


    # SHAP explanation
    explainer = shap.LinearExplainer(model,
                                  vectorizer.transform(vectorizer.get_feature_names_out()),
                                  feature_perturbation="interventional")
    shap_values = explainer.shap_values(X_input)

    # Get feature names and shap values
    feature_names = vectorizer.get_feature_names_out()
    dense_input = X_input.toarray()[0]
    indices = np.where(dense_input != 0)[0]
    tokens = feature_names[indices]
    shap_vals = shap_values[0][indices]

    # Print top contributing tokens
    print("\n📝 SHAP Explanation Summary:")
    positive_words = []
    negative_words = []

    for word, val in sorted(zip(tokens, shap_vals), key=lambda x: -abs(x[1])):
        if val > 0:
            positive_words.append(f'"{word}" (supports phishing)')
        elif val < 0:
            negative_words.append(f'"{word}" (supports legitimate)')
    if positive_words:
        print("The following words contributed to predicting this email as **phishing**:")
        print(" → " + ", ".join(positive_words))

    if negative_words:
        print("\nThe following words pushed the model toward predicting **legitimate**:")
        print(" → " + ", ".join(negative_words))

    print("\n📊 The chart below shows the top contributing words ranked by their SHAP values,")
    print("with red bars indicating words pushing the prediction toward **phishing**,")
    print("and blue bars indicating words supporting **legitimate**.")

    # Optional: plot
    plt.figure()  # 显式创建新图，否则可能出错
    shap.plots.bar(shap.Explanation(values=shap_vals, feature_names=tokens), max_display=10, show=False)  # 关键：show=False 不自动弹出图
    plt.savefig(f"D:\GITHUB\Summer-2025-ECE-597-Group11\data\picture\shap_bar_plot_{uid}.png", bbox_inches="tight")     # 保存到磁盘
    plt.close()  # 关闭图，防止 Streamlit 或 Jupyter 出现重叠图

<>:66: SyntaxWarning: invalid escape sequence '\G'
<>:66: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_14264\4140496829.py:66: SyntaxWarning: invalid escape sequence '\G'
  plt.savefig(f"D:\GITHUB\Summer-2025-ECE-597-Group11\data\picture\shap_bar_plot_{uid}.png", bbox_inches="tight")     # 保存到磁盘
c:\Users\yck\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import uuid

test_with_shap(f'model_tfidf_lr.pkl',f'tfidf_vectorizer_lr.pkl',uid = str(uuid.uuid4()))
# test_with_shap(f'naive_bayes_model_v1.pkl',f'tfidf_vectorizer_v1.pkl')


Prediction: 1 | Phishing Confidence: 0.6624


c:\Users\yck\miniconda3\Lib\site-packages\shap\explainers\_linear.py:99: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)



📝 SHAP Explanation Summary:
The following words contributed to predicting this email as **phishing**:
 → "click" (supports phishing), "immediately" (supports phishing)

The following words pushed the model toward predicting **legitimate**:
 → "update" (supports legitimate), "account" (supports legitimate), "verify" (supports legitimate), "important" (supports legitimate)

📊 The chart below shows the top contributing words ranked by their SHAP values,
with red bars indicating words pushing the prediction toward **phishing**,
and blue bars indicating words supporting **legitimate**.


In [12]:
import pandas as pd
df = pd.read_csv("D:\GITHUB\Summer-2025-ECE-597-Group11\data\kaggle\CEAS_08.csv")




<>:2: SyntaxWarning: invalid escape sequence '\G'
<>:2: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_10500\4255117661.py:2: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv("D:\GITHUB\Summer-2025-ECE-597-Group11\data\kaggle\CEAS_08.csv")


In [19]:
sample_list = df[df['label'] == 1][['subject', 'body']].head(30).to_dict(orient='records')


In [20]:
sample_list

[{'subject': 'Never agree to be a loser',
  'body': 'Buck up, your troubles caused by small dimension will soon be over!\nBecome a lover no woman will be able to resist!\nhttp://whitedone.com/\n\n\ncome. Even as Nazi tanks were rolling down the streets, the dreamersphilosopher or a journalist. He was still not sure.I do the same.'},
 {'subject': 'Befriend Jenna Jameson',
  'body': '\nUpgrade your sex and pleasures with these techniques http://www.brightmade.com\n\n\n'},
 {'subject': 'CNN.com Daily Top 10',
  'body': '>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= >THE DAILY TOP 10 >from CNN.com >Top videos and stories as of: Aug  1, 2008  3:58 PM EDT >+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+= TOP 10 VIDEOS 1. MONTAUK \'MONSTER\' http://www.cnn.com/video/partners/email/index.html?url=/video/us/2008/07/31/moos.montauk.monster.cnn Is it a devil dog? Is it a turtle? Is it the Montauk Monster? CNN\'s Jeanne Moos asks, "what is this thing?" 2. RACY PHOTOS OF